In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Importing the datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dataset= pd.read_csv('/content/drive/MyDrive/fraudTrain.csv')
test_dataset=pd.read_csv('/content/drive/MyDrive/fraudTest.csv')

In [5]:
train_dataset.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [6]:
train_dataset.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [7]:
train_dataset.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [8]:
df=pd.concat([train_dataset,test_dataset],axis=0)

In [9]:
df['is_fraud'].value_counts()

is_fraud
0    1842743
1       9651
Name: count, dtype: int64

In [10]:
train_dataset['is_fraud'].value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

In [11]:
independant=train_dataset[['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long',]]
dependant=train_dataset['is_fraud']

Under sampling to balance the data

In [12]:
from imblearn.under_sampling import RandomUnderSampler
ros=RandomUnderSampler(random_state=52)
x_ros,y_ros=ros.fit_resample(independant,dependant)

In [13]:
x_ros.shape

(15012, 22)

In [14]:
y_ros.value_counts()

is_fraud
0    7506
1    7506
Name: count, dtype: int64

Creating csv file with undersampled data

In [15]:
#convert the undersampled arrays to a Pandas Dataframe
undersampled_data=pd.DataFrame(data=x_ros, columns=independant.columns)

#add the dependant variable as a new column to the DataFrame
undersampled_data['target']=y_ros

#replace 'undersampled_data.csv' with the desired filename
output_file='undersampled_data.csv'

undersampled_data.to_csv(output_file,index=False)

In [16]:
#to convert to numerical
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

col=['category','first','last','city','state','job','street','merchant','dob','trans_date_trans_time','gender','trans_num']
for i in col:
  train_dataset[i]=le.fit_transform(train_dataset[i]).astype(int)

In [17]:
df=pd.read_csv('undersampled_data.csv')
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,target
0,1068088,2020-03-19 11:58:08,4209696857872688515,fraud_Dach-Borer,grocery_net,86.04,Sarah,Bell,F,6963 Russell Causeway Suite 231,...,37.8274,-88.6235,1943,"Restaurant manager, fast food",1971-10-19,7b0cbd697294c1cbf472e1469ff65213,1363694288,38.688327,-89.428770,0
1,902745,2019-12-27 17:07:32,4464457352619,fraud_Botsford and Sons,home,91.90,Breanna,Rodriguez,F,118 Cabrera Springs Apt. 105,...,29.8826,-84.5964,217,Television production assistant,1990-01-24,9b401740094ae6cb398dfe6b8adfb8f9,1356628052,29.861355,-84.524527,0
2,407892,2019-07-04 02:32:39,4377338765909719,fraud_Kutch LLC,gas_transport,52.23,Nicholas,Mcmahon,M,2123 Jonathan Falls Suite 753,...,36.1150,-84.0438,4729,"Geologist, wellsite",1982-04-19,3110122b0c5a925dba086089aff7ba26,1341369159,36.238795,-83.974205,0
3,176471,2019-04-02 01:08:50,4471568287204,fraud_Will Ltd,food_dining,127.19,Dakota,Maldonado,M,369 Cochran Radial,...,36.4899,-79.4736,3402,Insurance underwriter,1927-10-24,baff386db1669f0262d3c89617bd9320,1333328930,36.496492,-78.872277,0
4,867124,2019-12-19 12:05:27,3523898249167098,fraud_Hickle Group,shopping_pos,9.96,Joshua,Carpenter,M,04975 Allison Shoal,...,40.8798,-73.9067,1382480,"Therapist, drama",1990-11-09,b1d717794848c5a56bc5df7b24c4d5e8,1355918727,40.241683,-73.005654,0


In [18]:
# Assuming df is your DataFrame
unique_counts = df.nunique()

# Display the number of unique values per column
print(unique_counts)

Unnamed: 0               15012
trans_date_trans_time    15005
cc_num                     979
merchant                   693
category                    14
amt                      11480
first                      351
last                       480
gender                       2
street                     979
city                       890
state                       51
zip                        966
lat                        964
long                       965
city_pop                   875
job                        493
dob                        964
trans_num                15012
unix_time                15005
merch_lat                15009
merch_long               15007
target                       2
dtype: int64


In [19]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

col=['merchant','category','first','last','gender','city','state','job','street','trans_num','merchant','trans_date_trans_time','dob']
for i in col:
  df[i]=le.fit_transform(df[i]).astype(int)

In [20]:
independant=df[['Unnamed: 0', 'cc_num', 'category',
       'amt', 'first', 'last', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'unix_time','merchant',
       'merch_lat', 'merch_long','dob','trans_date_trans_time','gender']]
dependant=df['target']

In [23]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,target
0,1068088,12382,4209696857872688515,120,3,86.04,292,23,0,690,...,37.8274,-88.6235,1943,395,482,7232,1363694288,38.688327,-89.428770,0
1,902745,10275,4464457352619,69,6,91.90,43,369,0,107,...,29.8826,-84.5964,217,458,806,9139,1356628052,29.861355,-84.524527,0
2,407892,5011,4377338765909719,358,2,52.23,256,280,1,209,...,36.1150,-84.0438,4729,213,663,2821,1341369159,36.238795,-83.974205,0
3,176471,2518,4471568287204,668,1,127.19,88,254,1,378,...,36.4899,-79.4736,3402,250,10,11003,1333328930,36.496492,-78.872277,0
4,867124,9910,3523898249167098,252,12,9.96,180,60,1,46,...,40.8798,-73.9067,1382480,465,822,10493,1355918727,40.241683,-73.005654,0


FEATURE SELECTION

In [24]:
cor=df.corr()
cor_target=abs(cor['target'])
relevant_features=cor_target[cor_target>0.06]
print(relevant_features)

category    0.133121
amt         0.608019
dob         0.071367
target      1.000000
Name: target, dtype: float64


In [25]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,target
0,1068088,12382,4209696857872688515,120,3,86.04,292,23,0,690,...,37.8274,-88.6235,1943,395,482,7232,1363694288,38.688327,-89.428770,0
1,902745,10275,4464457352619,69,6,91.90,43,369,0,107,...,29.8826,-84.5964,217,458,806,9139,1356628052,29.861355,-84.524527,0
2,407892,5011,4377338765909719,358,2,52.23,256,280,1,209,...,36.1150,-84.0438,4729,213,663,2821,1341369159,36.238795,-83.974205,0
3,176471,2518,4471568287204,668,1,127.19,88,254,1,378,...,36.4899,-79.4736,3402,250,10,11003,1333328930,36.496492,-78.872277,0
4,867124,9910,3523898249167098,252,12,9.96,180,60,1,46,...,40.8798,-73.9067,1382480,465,822,10493,1355918727,40.241683,-73.005654,0


In [26]:
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np

#Create the ExtraTreesClassifier model
model=ExtraTreesClassifier()

#fit the model on your data
model.fit(independant,dependant)

#get feature importances
importance = model.feature_importances_

#sort the feature importances in descending order
sort_indices = np.argsort(importance)[::-1]

#print feature importance
for i,index in enumerate(sort_indices):
  print(f"{i+1}. Feature: {independant.columns[index]}-Importance:{importance[index]}")

1. Feature: amt-Importance:0.3954757025348236
2. Feature: category-Importance:0.08022530936275507
3. Feature: unix_time-Importance:0.04751118236933304
4. Feature: Unnamed: 0-Importance:0.046668421041077365
5. Feature: trans_date_trans_time-Importance:0.046251117749065755
6. Feature: dob-Importance:0.03273768385491839
7. Feature: merch_long-Importance:0.026590739886857027
8. Feature: merch_lat-Importance:0.026076685039422967
9. Feature: last-Importance:0.02595399190366148
10. Feature: street-Importance:0.025893392851891964
11. Feature: city-Importance:0.025642656937236184
12. Feature: first-Importance:0.02548754110068209
13. Feature: job-Importance:0.02538701497268854
14. Feature: lat-Importance:0.023872260181933325
15. Feature: cc_num-Importance:0.023619146610465824
16. Feature: zip-Importance:0.02351297511783809
17. Feature: city_pop-Importance:0.022981597353212715
18. Feature: state-Importance:0.02290514120377496
19. Feature: long-Importance:0.02282118724440853
20. Feature: merchant-

In [27]:
# Load testing dataset
test_data = pd.read_csv('/content/drive/MyDrive/fraudTest.csv')  # Replace 'path_to_testing_data.csv' with the actual path to your testing CSV file
test_data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [28]:
independant=test_dataset[['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'unix_time','merchant',
       'merch_lat', 'merch_long',]]
dependant=test_dataset['is_fraud']

In [29]:
from imblearn.under_sampling import RandomUnderSampler
ros=RandomUnderSampler(random_state=45)
x_ros,y_ros=ros.fit_resample(independant,dependant)

In [30]:
y_ros.value_counts()

is_fraud
0    2145
1    2145
Name: count, dtype: int64

In [31]:
#convert the undersampled arrays to a Pandas Dataframe
test_undersampled_data=pd.DataFrame(data=x_ros, columns=independant.columns)

#add the dependant variable as a new column to the DataFrame
test_undersampled_data['target']=y_ros

#replace 'undersampled_data.csv' with the desired filename
output_file='test_undersampled_data.csv'

test_undersampled_data.to_csv(output_file,index=False)

In [32]:
test_data=pd.read_csv('undersampled_data.csv')

In [33]:
#to convert to numerical
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

col=['category','first','last','city','state','job','street','merchant','dob','trans_date_trans_time','gender','trans_num']
for i in col:
  test_data[i]=le.fit_transform(test_data[i]).astype(int)

Training the model

In [34]:
#test_data=test_data.drop(['gender','merchant','long'],axis=1)

train_dataset=train_dataset.drop(['gender','merchant','long'],axis=1)

In [35]:

# Separate features (X) and target variable (y) for training and testing datasets
y_train = train_dataset['is_fraud']
X_train = train_dataset.drop(columns=['is_fraud'])  # Replace 'target_column_name' with the name of your target variable column

y_test = test_data['target']
X_test = test_data.drop(columns=['target'])  # Replace 'target_column_name' with the name of your target variable column

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=79,criterion="entropy")
classifier=classifier.fit(X_train,y_train.values.ravel())

In [37]:
test_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,target
0,1068088,12382,4209696857872688515,120,3,86.04,292,23,0,690,...,37.8274,-88.6235,1943,395,482,7232,1363694288,38.688327,-89.428770,0
1,902745,10275,4464457352619,69,6,91.90,43,369,0,107,...,29.8826,-84.5964,217,458,806,9139,1356628052,29.861355,-84.524527,0
2,407892,5011,4377338765909719,358,2,52.23,256,280,1,209,...,36.1150,-84.0438,4729,213,663,2821,1341369159,36.238795,-83.974205,0
3,176471,2518,4471568287204,668,1,127.19,88,254,1,378,...,36.4899,-79.4736,3402,250,10,11003,1333328930,36.496492,-78.872277,0
4,867124,9910,3523898249167098,252,12,9.96,180,60,1,46,...,40.8798,-73.9067,1382480,465,822,10493,1355918727,40.241683,-73.005654,0


In [ ]:
y_pred=classifier.predict(X_test)
y_pred

In [38]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
